# Part 2: Transform, Clean, and Analyze

In [ ]:
# import libraries
import pandas as pd
import matplotlib as plt
import numpy as np
from pathlib import Path
import seaborn as sns

In [56]:
PROCESSED_DATA_PATH = 'data/processed'
TRANFORMED_DATA_PATH = 'data/transformed'

In [57]:
Path(TRANFORMED_DATA_PATH).mkdir(parents=True, exist_ok=True)

In [40]:
# Using Kaggle Dataset and API Data
kaggle_ai_job_market_df = pd.read_csv(f"{PROCESSED_DATA_PATH}/ai_job_market.csv")
api_ai_data_df = pd.read_csv(f"{PROCESSED_DATA_PATH}/processed_api_data_jobdataapi.csv")

## Data Quality and Transformation

### 1. Kaggle AI Job Market Data

#### Data Quality Assessment

In [41]:
kaggle_ai_job_market_df.head()

,job_id,company_name,industry,job_title,skills_required,experience_level,employment_type,location,salary_range_usd,posted_date,company_size,tools_preferred
0,1,Foster and Sons,Healthcare,Data Analyst,"NumPy, Reinforcement Learning, PyTorch, Scikit...",Mid,Full-time,"Tracybury, AR",92860-109598,2025-08-20,Large,"KDB+, LangChain"
1,2,"Boyd, Myers and Ramirez",Tech,Computer Vision Engineer,"Scikit-learn, CUDA, SQL, Pandas",Senior,Full-time,"Lake Scott, CU",78523-144875,2024-03-22,Large,"FastAPI, KDB+, TensorFlow"
2,3,King Inc,Tech,Quant Researcher,"MLflow, FastAPI, Azure, PyTorch, SQL, GCP",Entry,Full-time,"East Paige, CM",124496-217204,2025-09-18,Large,"BigQuery, PyTorch, Scikit-learn"
3,4,"Cooper, Archer and Lynch",Tech,AI Product Manager,"Scikit-learn, C++, Pandas, LangChain, AWS, R",Mid,Full-time,"Perezview, FI",50908-123743,2024-05-08,Large,"TensorFlow, BigQuery, MLflow"
4,5,Hall LLC,Finance,Data Scientist,"Excel, Keras, SQL, Hugging Face",Senior,Contract,"North Desireeland, NE",98694-135413,2025-02-24,Large,"PyTorch, LangChain"


In [42]:
kaggle_ai_job_market_df.info()

<class 'pandas.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   job_id            2000 non-null   int64
 1   company_name      2000 non-null   str  
 2   industry          2000 non-null   str  
 3   job_title         2000 non-null   str  
 4   skills_required   2000 non-null   str  
 5   experience_level  2000 non-null   str  
 6   employment_type   2000 non-null   str  
 7   location          2000 non-null   str  
 8   salary_range_usd  2000 non-null   str  
 9   posted_date       2000 non-null   str  
 10  company_size      2000 non-null   str  
 11  tools_preferred   2000 non-null   str  
dtypes: int64(1), str(11)
memory usage: 187.6 KB


In [43]:
print("Total Null Values:\n", kaggle_ai_job_market_df.isna().sum())

Total Null Values:
 job_id              0
company_name        0
industry            0
job_title           0
skills_required     0
experience_level    0
employment_type     0
location            0
salary_range_usd    0
posted_date         0
company_size        0
tools_preferred     0
dtype: int64


In [44]:
print("Total Duplicate rows: ", kaggle_ai_job_market_df.duplicated().sum())

Total Duplicate rows:  0


#### Transformation and Cleaning

In [46]:
# splitting salary range to min and max
kaggle_ai_job_market_df[['salary_min_usd', 'salary_max_usd']] = (
    kaggle_ai_job_market_df['salary_range_usd']
    .str.split('-', expand=True)
    .astype(float)
)
kaggle_ai_job_market_df.drop(['salary_range_usd'], axis=1, inplace=True)

In [50]:
# comverting posted_date column to datetime
kaggle_ai_job_market_df['posted_date'] = pd.to_datetime(
    kaggle_ai_job_market_df['posted_date'],
    format='%Y-%m-%d'
)

In [52]:
# summary of all numerical and datetime columns
kaggle_ai_job_market_df.describe()

,job_id,posted_date,salary_min_usd,salary_max_usd
count,2000.000000,2000,2000.000000,2000.000000
mean,1000.500000,2024-09-21 15:00:00,95374.602000,150705.401000
min,1.000000,2023-09-21 00:00:00,40044.000000,54162.000000
25%,500.750000,2024-03-20 18:00:00,68234.500000,120751.250000
50%,1000.500000,2024-09-26 00:00:00,95440.000000,151337.500000
75%,1500.250000,2025-03-16 00:00:00,123580.500000,182200.000000
max,2000.000000,2025-09-19 00:00:00,149975.000000,247578.000000
std,577.494589,NaN,31882.260505,41716.198452


In [53]:
# getting numerical, categorical, and datetime columns list
numerical_columns = kaggle_ai_job_market_df.select_dtypes(include=[np.number]).columns.tolist()
categorical_columns = kaggle_ai_job_market_df.select_dtypes(include=['object', 'category', 'string']).columns.tolist()
datetime_columns = kaggle_ai_job_market_df.select_dtypes(include=['datetime', 'datetime64[ns]', 'datetime64[ns, UTC]']).columns.tolist()

print("Numerical Columns:\n", numerical_columns)
print("\nCategorical Columns:\n", categorical_columns)
print("\nDatetime Columns:\n", datetime_columns)

Numerical Columns:
 ['job_id', 'salary_min_usd', 'salary_max_usd']

Categorical Columns:
 ['company_name', 'industry', 'job_title', 'skills_required', 'experience_level', 'employment_type', 'location', 'company_size', 'tools_preferred']

Datetime Columns:
 ['posted_date']


In [58]:
# save transformed data
kaggle_ai_job_market_df.to_csv(f"{TRANFORMED_DATA_PATH}/cleaned_kaggle_ai_job_data.csv")

#### Exploratory Analysis and Visualization